# Ingesting XML data using pyspark read and write

In [0]:
%run ../../config/project_config

In [0]:
from pyspark.sql.functions import current_timestamp, col

In [0]:
source_path = f"{VOLUME_LANDING}/"
target_table = f"{CATALOG_NAME}.{SCHEMA_BRONZE}.county_metrics_xml"
file_path = f"{source_path}County_time_series_chunk_4.xml"

In [0]:
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {CATALOG_NAME}.{SCHEMA_BRONZE}.county_metrics_xml (
        load_dt TIMESTAMP,
        source_file STRING
    )
    USING DELTA
""")

In [0]:
df_raw=spark.read \
    .format("xml") \
    .option("rootTag", "root") \
    .option("rowTag", "record") \
    .load(file_path)

In [0]:
df_processed = df_raw \
    .withColumn("load_dt", current_timestamp()) \
    .withColumn("source_file", col("_metadata.file_path"))

In [0]:
df_processed.write \
    .format("delta") \
    .mode("append") \
    .option("mergeSchema", "true") \
    .saveAsTable(target_table)